#**Import Statements + Creating Environment**

In [2]:
import numpy as np
import numpy.random as rand

In [3]:
class Cell():
    target = 0
    cell_type = ""
    prob_not_found = 0
    def __init__(self, cell_type, prob_not_found):
        self.cell_type = cell_type
        self.prob_not_found = prob_not_found
        

In [28]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")
    
env = [[] for i in range(dimension)] # create a list of empty lists
cell_type = ""
prob_not_found = 0
for i in range(dimension):
    for j in range(dimension):
        p = rand.random()
        if p < 0.25:
            cell_type = "flat"
            prob_not_found = 0.1
        elif p <0.50:
            cell_type = "hilly"
            prob_not_found = 0.3
        elif p <0.75:
            cell_type = "forested"
            prob_not_found = 0.7
        else:
            cell_type = "maze of caves"
            prob_not_found = 0.9
        env[i].append(Cell(cell_type, prob_not_found))

Enter dimension of board: 5


In [29]:
#set target in environment
env[3][0].target = 1

In [31]:
print(env[3][0].target)

1


# **Helper Functions**

In [7]:
def search_cell(current_position):
    x = current_position[0]
    y = current_position[1]
    
    if env[x][y].target == 1:
        p = rand.random()
        if p > env[x][y].prob_not_found:
            return True
    return False

In [8]:
#variable assignment tree
def bayes_theorem(current_position, updating_position, belief):
    x = current_position[0]
    y = current_position[1]
    i = updating_position[0]
    j = updating_position[1]
    
    numerator = belief[i][j]
    #print(f"numerator: {numerator}")
    denominator = (1 - belief[x][y]) + belief[x][y]*env[x][y].prob_not_found
    #print(f"denominator: {denominator}")
    return numerator/denominator 

In [9]:
def bayes_current(current_position, belief):
    x = current_position[0]
    y = current_position[1]
    
    numerator = belief[x][y] * env[x][y].prob_not_found
    denominator = (1 - belief[x][y]) + belief[x][y]*env[x][y].prob_not_found
    return numerator/denominator 

In [10]:
def get_valid_neighbors(current_node):
  get_neighbors = [] #list of possible neighbors
  for i in range(-1,2):
    for j in range(-1,2):
      if i==0 and j==0:
        continue
      get_neighbors.append((i,j))
  x, y = current_node
  neighbors = set([])
  for i in get_neighbors:
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]

    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    neighbors.add((neighbor_x, neighbor_y))
  return neighbors

# **Basic Agent 1**

In [35]:
def basic_1(dimension, env):
  total_distance = 0
  belief = np.zeros((dimension,dimension))
  for i in range(dimension):
      for j in range(dimension):
          belief[i][j] = 1/(dimension*dimension)
  #print(belief)
  searching = True
  current_position = [np.random.randint(0,dimension),np.random.randint(0,dimension)]
  count = 1
  while searching:
      #print(f"current_position: {current_position}")
      #search cell
      query = search_cell(current_position)
      if query:
          #print(f"Found target at {current_position}")
          searching = False
          break
      count += 1    
      #update belief
      for i in range(dimension):
          for j in range(dimension):
              if i == current_position[0] and j == current_position[1]:
                  continue
              #Bayes Theorem
              calculated_belief = bayes_theorem(current_position, (i,j), belief)
              belief[i][j] = calculated_belief #update belief
      belief[current_position[0]][current_position[1]] = bayes_current(current_position, belief)
      #print(belief)
      
      #choose next position
      max_belief = 0
      total_belief = belief[current_position[0]][current_position[1]]
      for i in range(dimension):
          for j in range(dimension):
              total_belief += belief[i][j]
              if belief[i][j] > max_belief:
                  max_belief = belief[i][j]
                  new_position = [i,j]
                  distance = abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])
              if belief[i][j] == max_belief:
                if (abs(current_position[0] - i) + abs(current_position[1] - j)) > (abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])):
                  new_position = [i,j]
                  distance = abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])
      current_position = new_position
      total_distance += distance
      #print(f"sum of belief: {total_belief}")
      #print(f"sum of belief: {sum(sum(belief))}")
      #print()
  return total_distance,count

In [38]:
distance,searches = basic_1(dimension, env)
print(distance)
print(searches)

65
11


# **Basic Agent 2**

In [39]:
def basic_2(dimension, env):
  total_distance = 0
  belief = np.zeros((dimension,dimension))
  for i in range(dimension):
      for j in range(dimension):
          belief[i][j] = 1/(dimension*dimension)
  #print(belief)
  searching = True
  current_position = [np.random.randint(0,dimension),np.random.randint(0,dimension)]
  searches = 1
  while searching:
      #print(f"current_position: {current_position}")
      #search cell
      query = search_cell(current_position)
      if query:
          #print(f"Found target at {current_position}")
          searching = False
          break
      searches += 1   
      #update belief
      for i in range(dimension):
          for j in range(dimension):
              if i == current_position[0] and j == current_position[1]:
                  continue
              #Bayes Theorem
              calculated_belief = bayes_theorem(current_position, (i,j), belief)
              belief[i][j] = calculated_belief #update belief
      belief[current_position[0]][current_position[1]] = bayes_current(current_position, belief)
      #print(belief)
      
      #choose next position
      max_belief = 0
      total_belief = belief[current_position[0]][current_position[1]] *(1-env[current_position[0]][current_position[1]].prob_not_found)
      for i in range(dimension):
          for j in range(dimension):
              total_belief += belief[i][j] * (1 - env[i][j].prob_not_found)
              if belief[i][j]*(1 - env[i][j].prob_not_found) > max_belief:
                  max_belief = belief[i][j]*(1 - env[i][j].prob_not_found)
                  new_position = [i,j]
                  distance = abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])
              if belief[i][j]*(1 - env[i][j].prob_not_found) == max_belief:
                if (abs(current_position[0] - i) + abs(current_position[1] - j)) > (abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])):
                  new_position = [i,j]
                  distance = abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])
      current_position = new_position
      total_distance += distance
      #print(f"sum of belief: {total_belief}")
      #print(f"sum of belief: {sum(sum(belief))}")
      #print()
  return total_distance,searches

In [42]:
path = basic_2(dimension,env)
print(path)

(53, 11)


# **Improved Agent**

In [46]:
def improved_agent(dimension, env):
  total_distance = 0
  belief = np.zeros((dimension,dimension))
  belief_find = np.zeros((dimension,dimension))
  for i in range(dimension):
      for j in range(dimension):
          belief[i][j] = 1/(dimension*dimension)
  for i in range(dimension):
      for j in range(dimension):
          belief_find[i][j] = 1/(dimension*dimension) * (1-env[i][j].prob_not_found)
  a= sum(sum(belief_find))
  for i in range(dimension):
      for j in range(dimension):
          belief_find[i][j] = belief_find[i][j]/a
  #print(belief)
  #print(belief_find)
  searching = True
  current_position = [np.random.randint(0,dimension),np.random.randint(0,dimension)]
  count = 1 
  while searching:
      #print(f"current_position: {current_position}")
      #search cell
      query = search_cell(current_position)
      if query:
          #print(f"Found target at {current_position}")
          searching = False
          break
      count += 1
      #update belief
      for i in range(dimension):
          for j in range(dimension):
              if i == current_position[0] and j == current_position[1]:
                  continue
              #Bayes Theorem
              calculated_belief = bayes_theorem(current_position, (i,j), belief)
              belief[i][j] = calculated_belief #update belief
              belief_find[i][j] = belief[i][j]*(1-env[i][j].prob_not_found) #update belief_find

      belief[current_position[0]][current_position[1]] = bayes_current(current_position, belief)
      belief_find[current_position[0]][current_position[1]] = belief[current_position[0]][current_position[1]]*(1-env[current_position[0]][current_position[1]].prob_not_found)
      for i in range(dimension):
          for j in range(dimension):
            belief_find[i][j] = belief_find[i][j]/sum(sum(belief_find))
      #print(belief)

      #choose next position
      max_belief = 0
      total_belief = belief[current_position[0]][current_position[1]] + belief_find[current_position[0]][current_position[1]]
      for i in range(dimension):
          for j in range(dimension):
              total_belief += (belief[i][j] + belief_find[i][j])
              if (belief[i][j] + belief_find[i][j]) > max_belief:
                  max_belief = (belief[i][j] + belief_find[i][j])
                  new_position = [i,j]
                  distance = abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])
              if (belief[i][j] + belief_find[i][j]) == max_belief:
                if (abs(current_position[0] - i) + abs(current_position[1] - j)) > (abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])):
                  new_position = [i,j]
                  distance = abs(new_position[0] - current_position[0]) + abs(new_position[1] - current_position[1])
      current_position = new_position
      total_distance += distance
      #print(f"sum of belief: {total_belief}")
      #print(f"sum of belief: {sum(sum(belief))}")
      #print(f"sum of belief: {sum(sum(belief_find))}")
      #print()

  return total_distance,count

In [51]:
path = improved_agent(dimension, env)
print(path)

(11, 3)


# **Agent Averages**

**Average for Basic Agent 1**

In [44]:
num_searches = []
distance_list = []
for i in range(30):
  dimension = 10
  env = [[] for i in range(dimension)] # create a list of empty lists
  cell_type = ""
  prob_not_found = 0
  for i in range(dimension):
      for j in range(dimension):
          p = rand.random()
          if p < 0.25:
              cell_type = "flat"
              prob_not_found = 0.1
          elif p <0.50:
              cell_type = "hilly"
              prob_not_found = 0.3
          elif p <0.75:
              cell_type = "forested"
              prob_not_found = 0.7
          else:
              cell_type = "maze of caves"
              prob_not_found = 0.9
          env[i].append(Cell(cell_type, prob_not_found))
  x = np.random.randint(0,dimension)
  y = np.random.randint(0,dimension)
  env[x][y].target = 1
  #print((x,y))
  distances,searches = basic_1(dimension, env)
  num_searches.append(searches)
  distance_list.append(distances)

average_num_searches_basic_agent_1 = sum(num_searches)/len(num_searches) 
average_distance_basic_agent_1 = sum(distance_list)/len(distance_list) 
#print()
print(average_num_searches_basic_agent_1)
print(average_distance_basic_agent_1)

287.7
2597.366666666667


**Average for Basic Agent 2**

In [45]:
num_searches = []
distance_list = []
for i in range(30):
  dimension = 10
  env = [[] for i in range(dimension)] # create a list of empty lists
  cell_type = ""
  prob_not_found = 0
  for i in range(dimension):
      for j in range(dimension):
          p = rand.random()
          if p < 0.25:
              cell_type = "flat"
              prob_not_found = 0.1
          elif p <0.50:
              cell_type = "hilly"
              prob_not_found = 0.3
          elif p <0.75:
              cell_type = "forested"
              prob_not_found = 0.7
          else:
              cell_type = "maze of caves"
              prob_not_found = 0.9
          env[i].append(Cell(cell_type, prob_not_found))
  x = np.random.randint(0,dimension)
  y = np.random.randint(0,dimension)
  env[x][y].target = 1
  #print((x,y))
  distances,searches = basic_2(dimension, env)
  num_searches.append(searches)
  distance_list.append(distances)

average_num_searches_basic_agent_2 = sum(num_searches)/len(num_searches) 
average_distance_basic_agent_2 = sum(distance_list)/len(distance_list) 
#print()
print(average_num_searches_basic_agent_2)
print(average_distance_basic_agent_2)

177.7
1649.8333333333333


**Average for Improved Agent**

In [52]:
num_searches = []

for i in range(30):
  dimension = 10
  env = [[] for i in range(dimension)] # create a list of empty lists
  cell_type = ""
  prob_not_found = 0
  for i in range(dimension):
      for j in range(dimension):
          p = rand.random()
          if p < 0.25:
              cell_type = "flat"
              prob_not_found = 0.1
          elif p <0.50:
              cell_type = "hilly"
              prob_not_found = 0.3
          elif p <0.75:
              cell_type = "forested"
              prob_not_found = 0.7
          else:
              cell_type = "maze of caves"
              prob_not_found = 0.9
          env[i].append(Cell(cell_type, prob_not_found))
  x = np.random.randint(0,dimension)
  y = np.random.randint(0,dimension)
  env[x][y].target = 1
  #print((x,y))
  distances,searches = improved_agent(dimension, env)
  num_searches.append(searches)
  distance_list.append(distances)

average_num_searches_improved_agent = sum(num_searches)/len(num_searches) 
average_distance_improved_agent = sum(distance_list)/len(distance_list) 
#print()
print(average_num_searches_improved_agent)
print(average_distance_improved_agent)

159.4
1220.2833333333333
